In [1]:
# imports
import math
import mmh3
from bitarray import bitarray
from random import shuffle

# <center> Bloom Filters - The Art of Searching Millions of Usernames to Return "The username is taken. Try another" </center>



Suppose you are creating an account on Gmail, Facebook or Twitter you want to enter a cool username, you entered it and got a message, “Username is already taken”. You added your birth date along username, still no luck. Now you have added your university roll number also, still got “Username is already taken”. It’s really frustrating, isn’t it?
But have you ever thought how quickly Gmail checks availability of username by searching millions of username registered with it. There are many ways to do this job:

  - <b>Linear search</b>: Bad idea! Linear search is rarely practical because other search algorithms and schemes, such as the binary search algorithm and hash tables, allow significantly faster searching.

    Let’s assume that Facebook stores all the data in its directory. And the software simply checks each name on the list one at a time and if it doesn’t find a match, it tells you your desired username is available.
    Doesn’t sound sensible, does it?
    The software has to look at each name every time a username needs to be verified. The technique is unreasonable when you compare it with the Facebook database, which has over 1.5 billion users, and Twitter, which has 300 million users.
    
    
  - <b>Binary search</b>: Store all username alphabetically and compare entered username with middle one in list. If it matched, then username is taken otherwise figure out, whether entered username will come before or after middle one and if it will come after, neglect all the usernames before middle one(inclusive). Now search after middle one and repeat this process until you got a match or search end with no match. This technique is better and promising but still it requires multiple steps.
  
    In other view:
    Facebook keeps all the data sorted and arranged in an alphabetized list. The list is 1.5 billion characters long, stored like a, aa,aaa……xyy, XYZ, yaa,yaa,yxz, zaa, zac and is very similar to your dictionary. When you enter a name, it matches the entry with the username exactly in the middle of the list. If it matches, the software rejects the new username. If it doesn’t match (which has a lot of possibilities), the next question the software addresses are “ If searched alphabetically, does the requested username come before or after this username in the middle?” If it comes before, then the software knows that all the 750 million people after the username found in the middle of the list is of no use for the current search. That eliminates 750 million possibilities in a single comparison. If the desired username comes after the name in the middle (alphabetically), it eliminates all the names before it. Either way, the software eliminates almost 750 million names for search in the first comparison. Next, it takes the selected half of the list and immediately matches the requested username with the name in the middle of the remaining list. If it matches, the requested name is rejected and if it doesn’t, the requested name is again checked for the possibility of it occurring before or after the name in the middle. If it is before, reject the 350 million names after the name. And go ahead with divide and conquer for the rest of the names as done earlier. If the requested name is after the middle string, reject the names before it and try with the 350 million remaining names. By dividing the list every time, you can compare the required username with the names in the list quite quickly…
    
But, There must be something better!!

  - <b>Bloom Filter</b> is a data structure that can do this job.

For understanding bloom filters, you must know what are <b>data structures</b> and what is <b>hashing</b>. 
So, let's take a quick look at them.
  



### Data Structures and Where to Find Them

In computer science, a data structure is a data organization, management, and storage format that enables efficient access and modification. More precisely, a data structure is a collection of data values, the relationships among them, and the functions or operations that can be applied to the data.

Data structures are generally based on the ability of a computer to fetch and store data at any place in its memory, specified by a pointer—a bit string, representing a memory address, that can be itself stored in memory and manipulated by the program. Thus, the array and record data structures are based on computing the addresses of data items with arithmetic operations, while the linked data structures are based on storing addresses of data items within the structure itself.

The implementation of a data structure usually requires writing a set of procedures that create and manipulate instances of that structure. The efficiency of a data structure cannot be analyzed separately from those operations. This observation motivates the theoretical concept of an abstract data type, a data structure that is defined indirectly by the operations that may be performed on it, and the mathematical properties of those operations (including their space and time cost).

There are numerous types of data structures, generally built upon simpler primitive data types. Ley see the most commonly used data structures:
 * <i>Array</i> - A structure of fixed-size, which can hold items of the same data type. It can be an array of integers, an array of floating-point numbers, an array of strings or even an array of arrays (such as 2-dimensional arrays). Arrays are indexed, meaning that random access is possible.
 * <i>Linked Lists</i> - A linked list is a sequential structure that consists of a sequence of items in linear order which are linked to each other. Hence, you have to access data sequentially and random access is not possible. Linked lists provide a simple and flexible representation of dynamic sets.
 * <i>Record</i> - A record (also called tuple or struct) is an aggregate data structure. A record is a value that contains other values, typically in fixed number and sequence and typically indexed by names. The elements of records are usually called fields or members.
 * <i>Object</i> - An object is a data structure that contains data fields, like a record does, as well as various methods which operate on the data contents. An object is an in-memory instance of a class from a taxonomy. In the context of object-oriented programming, records are known as plain old data structures to distinguish them from objects.
 * <i>Stack</i> - A stack is a LIFO (Last In First Out — the element placed at last can be accessed at first) structure which can be commonly found in many programming languages. This structure is named as “stack” because it resembles a real-world stack — a stack of plates.
 * <i>Queues</i> - A queue is a FIFO (First In First Out — the element placed at first can be accessed at first) structure which can be commonly found in many programming languages. This structure is named as “queue” because it resembles a real-world queue — people waiting in a queue.
 * <i>Trees</i> - A tree is a hierarchical structure where data is organized hierarchically and are linked together. This structure is different from a linked list whereas, in a linked list, items are linked in a linear order.
 * <i>Heaps</i> - A Heap is a special case of a binary tree where the parent nodes are compared to their children with their values and are arranged accordingly.
 * <i>Graphs</i> - A graph consists of a finite set of vertices or nodes and a set of edges connecting these vertices. The order of a graph is the number of vertices in the graph. The size of a graph is the number of edges in the graph. Two nodes are said to be adjacent if they are connected to each other by the same edge.
 
 There are also some more complex structures and we are going to meet two of them up close.
 
 
 * <i>Probabilistic data structures</i> - Probabilistic data structures are a group of data structures that are extremely useful for big data and streaming applications. Generally speaking, these data structures use hash functions to randomize and compactly represent a set of items. Collisions are ignored but errors can be well-controlled under certain threshold. Comparing with error-free approaches, these algorithms use much less memory and have constant query time. They usually support union and intersection operations and therefore can be easily parallelized.
 * <i>Hash Tables</i> - A Hash Table is a data structure that stores values which have keys associated with each of them.



### Hash Tables and Functions and How to Tame Them

In order to understand the <i>Bloom filter</i>, we need to have some basic knowledge about <i>Hash Tables</i> and <i>Hash Functions</i>.

Direct Addressing uses the one-to-one mapping between the values and keys when storing in a table. However, there is a problem with this approach when there is a large number of key-value pairs. The table will be huge with so many records and may be impractical or even impossible to be stored, given the memory available on a typical computer. To avoid this issue we use <i>hash tables</i>.

As we described above, the  <b>Hash Table</b> is a data structure that stores values which have keys associated with each of them. Furthermore, it supports lookup efficiently if we know the key associated with the value. Hence it is very efficient in inserting and searching, irrespective of the size of the data.

A special function named as the <b>hash function ($h$)</b> is used to overcome the aforementioned problem in direct addressing.
In direct accessing, a value with key <b>$k$</b> is stored in the slot <b>$k$</b>. Using the hash function, we calculate the index of the table (slot) to which each value goes. The value calculated using the hash function for a given key is called the <b>hash value</b> which indicates the index of the table to which the value is mapped.

$$h(k) = k\%m$$

* $h$ - Hash function
* $k$ - Key of which the hash value should be determined
* $m$ - Size of the hash table (number of slots available). A prime value that is not close to an exact power of 2 is a good choice for $m$.

<img src="./images/hash_table.png">

> <center><em> Fig 1. Representation of a Hash Function </em></center>

Consider the hash function $h(k) = k\%20$, where the size of the hash table is 20. Given a set of keys, we want to calculate the hash value of each to determine the index where it should go in the hash table. Consider we have the following keys, the hash and the hash table index.

$1 → 1\%20 → 1$

$5 → 5\%20 → 5$

$23 → 23\%20 → 3$

$63 → 63\%20 → 3$

From the last two examples given above, we can see that collision can arise when the hash function generates the same index for more than one key. We can resolve collisions by selecting a suitable hash function $h$ and use techniques such as <b>chaining</b> and <b>open addressing</b>. We are going to learn more for these techniques in the next project.

### Accuracy: True vs. False Positive/Negative

Before we meet the Bloom filters, there is something more that had to be described.

<img src="./images/positive-negative-true-false-matrix.png">

As demonstrated in the featured image, a model’s individual predictions can either be true or false meaning the model is right or wrong. The actual value of the data point is also important. You may wonder why we need a model that makes predictions if we know the actual values. Here, we are referring to the model’s performance on the training data, data where we know the answers. Actual value of the data points can either be the values we are trying to identify in the dataset (positives) or other values (negatives). Therefore the 4 possible results of a model’s individual predictions are:

 * <b>True positive</b>: The prediction is correct and the actual value is positive (i.e. this value is one of the values that the model was trying to identify. In case of a model searching to identify customers who are likely to buy the product, this data point represents a potential buyer)
 * <b>False positive</b>: The prediction is wrong and the actual value is positive
 * <b>True negative</b>: The prediction is correct and the actual value is negative (i.e. this value is not one of the values that the model was trying to identify. In case of a model searching to identify customers who are likely to buy the product, this data point represents a customer who is not interested in buying)
 * <b>False negative</b>: The prediction is wrong and the actual value is negative
 
That's it. We are no longer completely ignorant in this area.

### What is Bloom Filter?

A Bloom filter is <b>a space-efficient probabilistic data structure</b> that is used to test whether an element is a member of a set. For example, checking availability of username is set membership problem, where the set is the list of all registered username. The price we pay for efficiency is that it is probabilistic in nature that means, there might be some False Positive results. <b>False positive means, it might tell that given username is already taken but actually it’s not.</b>

#### Lets see some interesting properties of Bloom Filters  

Unlike a standard hash table, a Bloom filter of a fixed size can represent a set with an arbitrarily large number of elements.

Adding an element never fails. However, the false positive rate increases steadily as elements are added until all bits in the filter are set to 1, at which point all queries yield a positive result.

Bloom filters never generate <b>false negative</b> result, i.e., telling you that a username doesn’t exist when it actually exists.

Deleting elements from filter is not possible because, if we delete a single element by clearing bits at indices generated by k hash functions, it might cause deletion of few other elements. Example – if we delete “geeks” (in given example below) by clearing bit at 1, 4 and 7, we might end up deleting “nerd” also Because bit at index 4 becomes 0 and bloom filter claims that “nerd” is not present.


### But how does this Bloom filter work?


A empty bloom filter is a <b>bit array</b> of $m$ bits, all set to zero, like this:

<img src="./images/enpty_bit_array-300x56.png" style="min-height: 70px" alt="Image of empty bloom filter">

>  <center><em>Fig. 2 - Empty bloom filter</em></center>

We need $k$ number of hash functions to calculate the hashes for a given input. When we want to add an item in the filter, the bits at $k$ indices $h1(x)$, $h2(x)$, … $hk(x)$ are set, where indices are calculated using hash functions. 

<img src="./images/Probabilistic1.png" style="min-height: 70px" alt="Image of empty bloom filter">

>  <center><em>Fig. 3 - Filling bloom filter</em></center>

<b>1. Example</b> – Suppose we want to enter “geeks” in the filter, we are using 3 hash functions and a bit array of length 10, all set to 0 initially. First we’ll calculate the hashes as following :  

$h1(“geeks”) \% 10 = 1$

$h2(“geeks”) \% 10 = 4$

$h3(“geeks”) \% 10 = 7$

<i>Note: These outputs are random for explanation only.</i> 

Now we will set the bits at indices 1, 4 and 7 to 1 

<img src="./images/geeks1-300x107.png" style="min-height: 130px" alt="Image of 'geeks' in the bloom filter">

>  <center><em>Fig. 4 - Set "geeks" in the the relevant positions</em></center>


<b>2. Example</b> - Again we want to enter “nerd”, similarly we’ll calculate hashes

$h1(“nerd”) \% 10 = 3$

$h2(“nerd”) \% 10 = 5$

$h3(“nerd”) \% 10 = 4$


Set the bits at indices 3, 5 and 4 to 1 

<img src="./images/nerd-300x114.png" style="min-height: 130px" alt="Image of 'nerd' in the bloom filter">

>  <center><em>Fig. 5 - Set "nerd" in the the relevant positions</em></center>

<i>Conclusion</i>

Now if we want to check “geeks” is present in filter or not. We’ll do the same process but this time in reverse order. We calculate respective hashes using $h1$, $h2$ and $h3$ and check if all these indices are set to 1 in the bit array. If all the bits are set then we can say that “geeks” is <b>probably present</b>. If any of the bit at these indices are 0 then “geeks” is <b>definitely not present</b>. 


### False Positive in Bloom Filters

The question is why we said <b>“probably present”</b>, why this uncertainty. Let’s understand this with an example. Suppose we want to check whether “cat” is present or not. We’ll calculate hashes using $h1$, $h2$ and $h3$.  

$h1(“cat”) \% 10 = 1$

$h2(“cat”) \% 10 = 3$

$h3(“cat”) \% 10 = 7$

If we check the bit array, bits at these indices are set to 1 but we know that “cat” was never added to the filter. Bit at index 1 and 7 was set when we added “geeks”in ex.1 and bit 3 was set we added “nerd” in ex.2. 

<img src="./images/cat-300x109.png" style="min-height: 130px" alt="Image of 'cat' in the bloom filter">

>  <center><em>Fig. 6 - Set "cat" in the the relevant positions</em></center>


So, because bits at calculated indices are already set by some other item, bloom filter erroneously claim that “cat” is present and generating a false positive result. Depending on the application, it could be huge downside or relatively okay.

But wait, I have good news!
We can control the probability of getting a false positive by controlling the size of the Bloom filter. More space means fewer false positives. If we want decrease probability of false positive result, we have to use more number of hash functions and larger bit array. This would add latency in addition of item and checking membership.

Operations that a Bloom Filter supports:
* <b>insert(x)</b> - To insert an element in the Bloom Filter.
* <b>lookup(x)</b> - to check whether an element is already present in Bloom Filter with a positive false probability.

NOTE : We cannot delete an element in Bloom Filter.

### Probability of False positivity 

  * Let $m$ be the size of bit array, $k$ be the number of hash functions and $n$ be the number of expected elements to be inserted in the filter, then the probability of false positive $P$ can be calculated as:

$$P=(1-[1-\frac{1}{m}]^{kn})^{k}$$

 * Size of Bit Array: If expected number of elements $n$ is known and desired false positive probability is $P$ then the size of bit array $m$ can be calculated as:

$$m=-\frac{n\ln P}{(ln2)^{2}}$$

 * Optimum number of hash functions: The number of hash functions $k$ must be a positive integer. If $m$ is size of bit array and $n$ is number of elements to be inserted, then $k$ can be calculated as:

$$k=\frac{m}{n} \ln 2$$

* Space Efficiency:
    If we want to store large list of items in a set for purpose of set membership, we can store it in hashmap, tries or simple array or linked list. All these methods require storing item itself, which is not very memory efficient. For example, if we want to store “geeks” in hashmap we have to store actual string “geeks” as a key value pair {some_key : ”geeks”}. 

    Bloom filters do not store the data item at all. As we have seen they use bit array which allow hash collision. Without hash collision, it would not be compact.  

### Choice of Hash Function

The hash function used in bloom filters should be independent and uniformly distributed. They should be fast as possible. Fast simple non cryptographic hashes which are independent enough include murmur, FNV series of hash functions and Jenkins hashes. 

Generating hash is major operation in bloom filters. Cryptographic hash functions provide stability and guarantee but are expensive in calculation. With increase in number of hash functions $k$, bloom filter become slow. All though non-cryptographic hash functions do not provide guarantee but provide major performance improvement.


In [2]:
# Basic implementation of Bloom Filter class in Python3

class BloomFilter(object):
 
    '''
    Class for Bloom filter, using murmur3 hash function
    '''
 
    def __init__(self, items_count, fp_prob):

        # False posible probability in decimal
        self.fp_prob = fp_prob
 
        # Size of bit array to use
        self.size = self.get_size(items_count, fp_prob)
 
        # number of hash functions to use
        self.hash_count = self.get_hash_count(self.size, items_count)
 
        # Bit array of given size
        self.bit_array = bitarray(self.size)
 
        # initialize all bits as 0
        self.bit_array.setall(0)
 
    def add(self, item):
        '''
        Add an item in the filter
        '''
        digests = []
        for i in range(self.hash_count):
 
            # create digest for given item.
            # work as seed to mmh3.hash() function
            # With different seed, digest created is different
            digest = mmh3.hash(item, i) % self.size
            digests.append(digest)
 
            # set the bit True in bit_array
            self.bit_array[digest] = True
 
    def check(self, item):
        '''
        Check for existence of an item in filter
        '''
        for i in range(self.hash_count):
            digest = mmh3.hash(item, i) % self.size
            if self.bit_array[digest] == False:
 
                # if any of bit is False then,its not present
                # in filter
                # else there is probability that it exist
                return False
        return True
 
    @classmethod
    def get_size(self, n, p):
        '''
        Return the size of bit array(m) to used using
        following formula
        m = -(n * lg(p)) / (lg(2)^2)
        n : int
            number of items expected to be stored in filter
        p : float
            False Positive probability in decimal
        '''
        m = -(n * math.log(p))/(math.log(2)**2)
        return int(m)
 
    @classmethod
    def get_hash_count(self, m, n):
        '''
        Return the hash function(k) to be used using
        following formula
        k = (m/n) * lg(2)
 
        m : int
            size of bit array
        n : int
            number of items expected to be stored in filter
        '''
        k = (m/n) * math.log(2)
        return int(k)

In [3]:
# Lets test the bloom filter.

n = 20 #no of items to add
p = 0.05 #false positive probability
 
bloomf = BloomFilter(n,p)
print("Size of bit array:{}".format(bloomf.size))
print("False positive Probability:{}".format(bloomf.fp_prob))
print("Number of hash functions:{}".format(bloomf.hash_count))
 
# words to be added
word_present = ['abound','abounds','abundance','abundant','accessable',
                'bloom','blossom','bolster','bonny','bonus','bonuses',
                'coherent','cohesive','colorful','comely','comfort',
                'gems','generosity','generous','generously','genial']
 
# word not added
word_absent = ['bluff','cheater','hate','war','humanity',
               'racism','hurt','nuke','gloomy','facebook',
               'geeksforgeeks','twitter']
 
for item in word_present:
    bloomf.add(item)
 
shuffle(word_present)
shuffle(word_absent)

test_words = word_present[:10] + word_absent
shuffle(test_words)
for word in test_words:
    if bloomf.check(word):
        if word in word_absent:
            print("'{}' is a false positive!".format(word))
        else:
            print("'{}' is probably present!".format(word))
    else:
        print("'{}' is definitely not present!".format(word))


Size of bit array:124
False positive Probability:0.05
Number of hash functions:4
'humanity' is definitely not present!
'comfort' is probably present!
'facebook' is definitely not present!
'abounds' is probably present!
'colorful' is probably present!
'war' is definitely not present!
'gloomy' is definitely not present!
'nuke' is definitely not present!
'bluff' is definitely not present!
'blossom' is probably present!
'hate' is definitely not present!
'hurt' is definitely not present!
'bonus' is probably present!
'cheater' is definitely not present!
'cohesive' is probably present!
'bolster' is probably present!
'generous' is probably present!
'abundant' is probably present!
'twitter' is a false positive!
'bonny' is probably present!
'racism' is definitely not present!
'geeksforgeeks' is definitely not present!


This scenario explains the concept of false positives used in Bloom filters. 

If we take a closer look at the returned data we could see <b>'twitter' is a false positive!</b>
This meens that 'twiter' was not included in the provided data, but was considered as posible to be there. The Bloom Filter cuts it, to insure that in is not included for sure. The absurd thing is that if you are not lucky, there is a 0.05 probability that your dream username will be rejected by mistake (as false positive) as 'twitter' was rejected above.

One can control the false positive by controlling the size of the Bloom filter.
More space is inversely proportional to false positives. We need to take a bit array of some specific size that is solely we need to assume for the system how much users will come into our system.

With technology and computers getting smarter and faster every day, even the brute-force method seems feasible.

But with space and time complexity, many companies, such as Reddit, prefer Binary search, whereas some others, such as Medium, use Bloom filters smartly to suggest articles for you without repeating them again on your timeline.

Bloom Filters is already used by a lot of databases like Cassandra, Postgres, HBase etc. and it is used by Google Chrome for detecting malicious websites. Basically it keeps all the malicious urls on chrome and before rendering and going to page it checks on this list with bloom filters and returns safety concerns if any.

The applications don't end there. The Bloom filter is used for Weak password detection, Internet Cache Protocol, Safe browsing in Google Chrome, Wallet synchronization in Bitcoin, Hash based IP Traceback, Cyber security like virus scanning and more.

Each of the above-mentioned techniques comes with its own advantages and disadvantages.

#### References
 * https://en.wikipedia.org/wiki/Data_structure
 * https://towardsdatascience.com/8-common-data-structures-every-programmer-must-know-171acf6a1a42
 * https://programming.guide/bloom-filter.html
 * https://en.wikipedia.org/wiki/Bloom_filter
 * https://blog.medium.com/what-are-bloom-filters-1ec2a50c68ff
 * https://www.quora.com/What-are-the-best-applications-of-Bloom-filters
 * https://www.geeksforgeeks.org/bloom-filters-introduction-and-python-implementation/
 * https://www.hackerearth.com/blog/developers/how-websites-check-username-availability-quickly/
 * https://medium.com/@rishabh171192/bloom-filters-aa58c66ebaf6
 * https://iq.opengenus.org/applications-of-bloom-filter/